In [9]:
#ИМПОРТ И ФУНКЦИИ С КЛАССАМИ
import os
import glob
import tqdm
import h5py
import torch
import random
import pickle
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torch.utils.data import Subset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from IPython.display import clear_output
from IPython.display import Audio, display
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

# выбираем устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#класс датасета
class AudioDataset(Dataset):
    def __init__(self, h5_path, smoothing=0.1):
        self.h5_path = h5_path
        self.file = h5py.File(h5_path, 'r')
        
        self.spectrograms = self.file['spectrograms']
        self.labels = self.file['labels']

        self.class_names = [name.decode('utf-8') for name in self.file.attrs['class_names']]
        self.num_classes = self.file.attrs['num_classes']
        
        print(f"Loaded {len(self.spectrograms)} samples from HDF5")
    
    def __len__(self):
        return len(self.spectrograms)
    
    def __getitem__(self, idx, smoothing=0.1):
        # Загружаем непосредственно из HDF5
        spectrogram = torch.tensor(self.spectrograms[idx], dtype=torch.float32)
        
        #smoothing labels
        # smooth_label = torch.tensor(self.labels[idx] * (1 - smoothing) + (1 - self.labels[idx]) * smoothing, dtype=torch.float32)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        return spectrogram, label
    
    def __del__(self):
        if hasattr(self, 'file'):
            self.file.close()
    
    def get_class_names(self):
        return self.class_names 
# для шумного датасета
class AsymmetricLoss(nn.Module):
    def __init__(self, gamma_pos=1, gamma_neg=4, clip=0.05, eps=1e-8):
        super(AsymmetricLoss, self).__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps

    def forward(self, x, y):
        # Логиты в вероятности
        x_sigmoid = torch.sigmoid(x)
        
        # Разделяем на позитивные и негативные
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Асимметричное отсечение: для негативных примеров ограничиваем снизу
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Асимметричное фокусирование
        los_pos = y * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=self.eps))
        
        # Разные gamma для позитивных и негативных
        if self.gamma_pos > 0:
            pt_pos = xs_pos * y + (1 - xs_pos) * (1 - y)         
            los_pos *= (1 - pt_pos) ** self.gamma_pos

        if self.gamma_neg > 0:
            pt_neg = xs_neg * (1 - y) + (1 - xs_neg) * y
            los_neg *= (1 - pt_neg) ** self.gamma_neg

        loss = los_pos + los_neg
        return -loss.mean()
# ещё вариант функции потерь
class MultiLabelAsymmetricLoss(nn.Module):
    def __init__(self, gamma_pos=1.0, gamma_neg=4.0, clip=0.05, 
                 label_smoothing=0.1, reduction='mean'):
        super(MultiLabelAsymmetricLoss, self).__init__()
        self.gamma_pos = gamma_pos
        self.gamma_neg = gamma_neg
        self.clip = clip
        self.label_smoothing = label_smoothing
        self.reduction = reduction
        
    def forward(self, logits, targets):
        # Label smoothing для мульти-лейбл
        if self.label_smoothing > 0:
            targets = targets * (1 - self.label_smoothing) + 0.5 * self.label_smoothing
        
        # Стандартная ASL логика
        probs = torch.sigmoid(logits)
        
        if self.clip is not None and self.clip > 0:
            probs = torch.clamp(probs, min=self.clip, max=1-self.clip)
            logits = torch.log(probs / (1 - probs))
        
        # Для мульти-лейбл: каждый класс независимо
        pos_loss = -targets * torch.log(probs + 1e-8)
        neg_loss = -(1 - targets) * torch.log(1 - probs + 1e-8)
        
        if self.gamma_pos > 0:
            pos_weight = (1 - probs) ** self.gamma_pos
            pos_loss = pos_weight * pos_loss
        
        if self.gamma_neg > 0:
            neg_weight = probs ** self.gamma_neg
            neg_loss = neg_weight * neg_loss
        
        loss = pos_loss + neg_loss
        
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss
# как вариант
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, inputs, targets):
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * bce_loss
        return focal_loss.mean()
#CNN + classifier    
class AudioCNN(nn.Module):
    def __init__(self, num_classes):
        
        super(AudioCNN, self).__init__()
        
        # CNN [1, 128, 256]
        self.conv_layers = nn.Sequential(
            
            # Блок 1
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),    #[16, 64, 128]
            nn.Dropout(0.2),
            
            # Блок 2
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),    #[32, 32, 64]
            nn.Dropout(0.25),
            
            # Блок 3 
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),    #[64, 16, 32]
            nn.Dropout(0.3),

            # Блок 4
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.35),

            nn.AdaptiveAvgPool2d((4,8)),    #[128, 4, 8]
        )

        # FC
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 8, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(1024, num_classes)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x
#AudioCNN + attention
class AudioCNN_2(nn.Module):
    def __init__(self, num_classes, hidden_size=512):
        super(AudioCNN_2, self).__init__()
        # CNN, вход - [1, 128, 256]
        self.conv_layers = nn.Sequential(
            
            # Блок 1
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=0), # [32, 124, 252] 
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.MaxPool2d(2), #[32, 62, 126]
            nn.Dropout(0.1),
            
            # Блок 2
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=0), #[64, 58, 122]
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.MaxPool2d(2), #[64, 29, 61]
            nn.Dropout(0.2),

            # Блок 3 
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=1), #[128, 26, 58]
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.MaxPool2d(2), #[128, 13, 29]
            nn.Dropout(0.3),

            # Блок 4 
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=0), #[256, 9, 25]
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.Dropout(0.4),

            nn.AdaptiveAvgPool2d((1, 2)), #[256, 1, 2]
        )

        # FC
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 1 * 2, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ELU(),
            nn.Dropout(0.5),

            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ELU(),
            nn.Dropout(0.4),

            nn.Linear(hidden_size // 2, num_classes)
        )

    #     # инициализация весов
    #     self._initialize_weights()

    # def _initialize_weights(self):
    #     for m in self.modules():
    #         if isinstance(m, nn.Conv2d):
    #             nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
    #             if m.bias is not None:
    #                 nn.init.constant_(m.bias, 0)
    #         elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
    #             nn.init.constant_(m.weight, 1)
    #             nn.init.constant_(m.bias, 0)
    #         elif isinstance(m, nn.Linear):
    #             nn.init.normal_(m.weight, 0, 0.01)
    #             nn.init.constant_(m.bias, 0)

    def forward(self, x):
        #CNN
        x = self.conv_layers(x)
        #FC
        x = self.classifier(x)
        return x

class AudioCNN_3(nn.Module):
    def __init__(self, num_classes, hidden_size=512):
        super(AudioCNN_3, self).__init__()
        # CNN, вход - [1, 128, 256]
        self.conv_1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=0), # [32, 124, 252] 
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.MaxPool2d(2), #[32, 62, 126]
            nn.Dropout(0.1),
        )

        self.conv_2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=0), # [64, 58, 122]
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.MaxPool2d(2), # [64, 29, 61]
            nn.Dropout(0.2),
        )

        self.conv_3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=1), # [128, 26, 58]
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.MaxPool2d(2), # [128, 13, 29]
            nn.Dropout(0.3),
        )

        self.conv_4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=0), # [256, 9, 25] 
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.Dropout(0.4),
        )
           
        self.avgpool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1))) #[256, 1, 1]

        # FC
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 1 * 1, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ELU(),
            nn.Dropout(0.5),

            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ELU(),
            nn.Dropout(0.5),

            nn.Linear(hidden_size // 2, num_classes)
        )    
    
    def forward(self, x):
        #CNN
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.avgpool(x)
        #FC
        x = self.classifier(x)
        return x

#полносвязная
class SimpleFCModel(nn.Module):
    def __init__(self, num_classes: int, input_height: int, input_width: int, hidden_size: int = 512):
        super(SimpleFCModel, self).__init__()
        
        self.input_height = input_height
        self.input_width = input_width
        self.num_classes = num_classes
        
        # Вычисляем размер после вытягивания в вектор
        self.flatten_size = input_height * input_width
        
        self.layers = nn.Sequential(
            # Вытягиваем в плоский вектор
            nn.Flatten(),
            
            # Первый скрытый слой
            nn.Linear(self.flatten_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),  # для регуляризации
            
            # Второй скрытый слой (можно добавить больше)
            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ReLU(), 
            nn.Dropout(0.5),
            
            # Выходной слой
            nn.Linear(hidden_size // 2, num_classes)
        )

    def forward(self, x):
        x = self.layers(x)
        return x
#метрики
def calculate_all_metrics(y_true, y_pred, loss, threshold):
    
    metrics = {}
    # Loss
    metrics['loss'] = loss
    # F1 Macro - баланс между классами (ОСНОВНАЯ)
    metrics['f1_macro'] = f1_score(y_true, y_pred, average='macro', zero_division=0)
    # Precision Macro - контроль false positive
    metrics['precision_macro'] = precision_score(y_true, y_pred, average='macro', zero_division=0)
    # Recall Macro - контроль false negative
    metrics['recall_macro'] = recall_score(y_true, y_pred, average='macro', zero_division=0)
    # Threshold - порог опредления label'а
    metrics['threshold'] = threshold

    # F1 для каждого класса
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)
    metrics['f1_per_class'] = f1_per_class
    
    return metrics

def train_epoch(model, loader, optimizer, criterion, epoch, num_epochs, threshold):
    model.train()
    all_targets = []
    all_predictions = []
    all_loss = 0.0
    
    # Создаем прогресс-бар
    pbar = tqdm.tqdm(loader, desc=f'Train >>> ', bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}') 

    for data, targets in pbar:
        data = data.to(device)
        targets = targets.to(device)   

        optimizer.zero_grad()                          
        outputs = model(data)                          
        loss = criterion(outputs, targets)             
        all_loss += loss.item()

        loss.backward()                                
        optimizer.step()                                 
        
        # Сохраняем предсказания и цели для метрик
        with torch.no_grad():            
            probs = torch.sigmoid(outputs)           
            predictions = (probs > threshold).float()            
            all_targets.append(targets.cpu())           
            all_predictions.append(predictions.cpu())
            
        # Обновляем прогресс-бар
        current_loss = all_loss / (pbar.n + 1)
        pbar.set_postfix({'Loss': f'{current_loss:.4f}'})

    all_targets = torch.cat(all_targets).numpy()
    all_predictions = torch.cat(all_predictions).numpy()
    all_loss /= len(loader)

    return calculate_all_metrics(all_targets, all_predictions, all_loss, threshold)

def evaluate_epoch(model, loader, criterion, epoch, num_epochs, threshold, auto_threshold):
    model.eval()
    all_targets = []
    all_predictions = []
    all_probs = []
    all_loss = 0.0

    pbar = tqdm.tqdm(loader, desc=f'  Val >>> ', bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}')

    with torch.no_grad():
        for data, targets in pbar:  
            data = data.to(device)
            targets = targets.to(device) 

            outputs = model(data)                       
            loss = criterion(outputs, targets)
            all_loss += loss.item()
            
            probs = torch.sigmoid(outputs)   
            predictions = (probs > threshold).float()   
            
            all_targets.append(targets.cpu())          
            all_predictions.append(predictions.cpu())
            all_probs.append(probs.cpu())

            # Обновляем прогресс-бар
            current_loss = all_loss / (pbar.n + 1)
            pbar.set_postfix({'Loss': f'{current_loss:.4f}'})
    
    all_targets = torch.cat(all_targets).numpy()
    all_predictions = torch.cat(all_predictions).numpy()
    all_probs = torch.cat(all_probs)
    all_loss /= len(loader)

    best_threshold = threshold
    best_f1 = 0.0
    
    # расчёт лучшего порога
    if auto_threshold:
        for threshold in torch.arange(0.1, 0.9, 0.1):
            predictions = (all_probs > threshold).float()
            f1 = f1_score(all_targets, predictions, average='macro')
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold.item()

    return calculate_all_metrics(all_targets, all_predictions, all_loss, best_threshold)

def train_model(model, train_loader, val_loader, optimizer, criterion, patience=10,
                num_epochs=50, threshold=0.5, save_dir='../data/models', output_str="",
                start_epoch=1, history=None, best_f1=0.0, auto_threshold=False):
    
    os.makedirs(save_dir, exist_ok=True)

    # Инициализация истории, если не передана
    if history is None:
        history = {
            'train_loss': [], 'val_loss': [],
            'train_f1': [], 'val_f1': [],
            'train_precision': [], 'val_precision': [],
            'train_recall': [], 'val_recall': [],
            'train_threshold': [], 'val_threshold': [],
            'epochs_completed': 0,
            'total_epochs': num_epochs,
            'best_f1': 0.0,
            'f1_per_class':[]
        }
    
    patience_counter = 0
    print_str = ""

    for epoch in range(start_epoch, start_epoch + num_epochs):

        clear_output(wait=False)

        print(output_str)

        print(print_str)
        print_str = ""
        
        print(f"Epoch {epoch}/{start_epoch + num_epochs - 1}")
        print("─" * 50)

        # Обучение и валидация
        train_metrics = train_epoch(model, train_loader, optimizer, criterion, epoch, start_epoch + num_epochs - 1, threshold)
        val_metrics = evaluate_epoch(model, val_loader, criterion, epoch, start_epoch + num_epochs - 1, threshold, auto_threshold)
        
        # Сохраняем историю
        history['train_loss'].append(train_metrics['loss'])
        history['val_loss'].append(val_metrics['loss'])
        history['train_f1'].append(train_metrics['f1_macro'])
        history['val_f1'].append(val_metrics['f1_macro'])
        history['train_precision'].append(train_metrics['precision_macro'])
        history['val_precision'].append(val_metrics['precision_macro'])
        history['train_recall'].append(train_metrics['recall_macro'])
        history['val_recall'].append(val_metrics['recall_macro'])
        history['epochs_completed'] = epoch
        history['best_f1'] = max(history.get('best_f1', 0.0), val_metrics['f1_macro'])
        history['f1_per_class'] = val_metrics['f1_per_class']

        # Выводим метрики
        print_str += f"Epoch {epoch}/{start_epoch + num_epochs - 1}\n"
        print_str += "─" * 50 + "\n"

        print_str += f"Train >>> Loss: {train_metrics['loss']:.4f} | F1: {train_metrics['f1_macro']:.4f} | "
        print_str += f"Precision: {train_metrics['precision_macro']:.4f} | Recall: {train_metrics['recall_macro']:.4f} | "
        print_str += f"Threshold: {threshold:.2f}\n"

        print_str += f"  Val >>> Loss: {val_metrics['loss']:.4f} | F1: {val_metrics['f1_macro']:.4f} | "
        print_str += f"Precision: {val_metrics['precision_macro']:.4f} | Recall: {val_metrics['recall_macro']:.4f} | "
        print_str += f"Threshold: {threshold:.2f}\n"
        print_str += "─" * 50 + "\n"

        # обновляем threshold
        threshold = val_metrics['threshold'] 

        # Early stopping и сохранение лучшей модели
        if val_metrics['f1_macro'] > best_f1:
            best_f1 = val_metrics['f1_macro']
            # Сохраняем полное состояние для продолжения обучения
            history_copy = history.copy()
            history_copy.pop('f1_per_class')
            checkpoint = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history_copy,
                'epoch': epoch,
                'best_f1': best_f1,
                'patience_counter': patience_counter,
                'threshold': threshold
            }
            torch.save(checkpoint, f'{save_dir}/best_model.pth')
            patience_counter = 0
            print_str += f"Новая лучшая модель сохранена! F1: {best_f1:.4f}\n"
        else:
            patience_counter += 1
            print_str += f"Нет улучшений: ({patience_counter}/{patience})\n"
            
        if patience_counter >= patience:
            print_str += f"Ранняя остановка на эпохе {epoch}."
            break

        # Сохраняем последнюю модель каждые N эпох
        if epoch % 5 == 0:
            checkpoint = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history_copy,
                'epoch': epoch,
                'best_f1': best_f1,
                'patience_counter': patience_counter,
                'threshold': threshold
            }
            torch.save(checkpoint, f'{save_dir}/checkpoint_epoch_{epoch}.pth')

    clear_output(wait=False)
    print(print_str) 

    return history, best_f1

def load_checkpoint(model, optimizer, checkpoint_path, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    history = checkpoint['history']
    start_epoch = checkpoint['epoch'] + 1
    best_f1 = checkpoint['best_f1']
    threshold = checkpoint.get('threshold', 0.5)
    
    load_str = f"Загружен чекпоинт эпохи {checkpoint['epoch']}\n" + f"Лучший F1: {best_f1:.4f}\n" + f"Всего эпох в истории: {len(history['train_loss'])}\n"
    
    return model, optimizer, history, start_epoch, best_f1, threshold, load_str

def plot_training_history(history, save_path=None):
    
    fig = plt.figure(figsize=(15, 14))
    fig.suptitle('История тренировки', fontsize=16, fontweight='bold')
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    gs = plt.GridSpec(3, 2, figure=fig)

    # 1. Loss
    ax1 = fig.add_subplot(gs[0,0])
    ax1.plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    ax1.plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    ax1.set_title('Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. F1 Score
    ax2 = fig.add_subplot(gs[0,1])
    ax2.plot(epochs, history['train_f1'], 'b-', label='Train F1', linewidth=2)
    ax2.plot(epochs, history['val_f1'], 'r-', label='Val F1', linewidth=2)
    ax2.set_title('F1 Macro Score')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('F1 Score')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Precision
    ax3 = fig.add_subplot(gs[1,0])
    ax3.plot(epochs, history['train_precision'], 'b-', label='Train Precision', linewidth=2)
    ax3.plot(epochs, history['val_precision'], 'r-', label='Val Precision', linewidth=2)
    ax3.set_title('Precision Macro')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Precision')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Recall
    ax4 = fig.add_subplot(gs[1,1])
    ax4.plot(epochs, history['train_recall'], 'b-', label='Train Recall', linewidth=2)
    ax4.plot(epochs, history['val_recall'], 'r-', label='Val Recall', linewidth=2)
    ax4.set_title('Recall Macro')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Recall')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Гистограмма F1 по классам (нижняя строка на всю ширину)
    ax5 = fig.add_subplot(gs[2, :])
    
    if history['f1_per_class'] is not None:
        f1_scores = history['f1_per_class']
        n_classes = len(f1_scores)
        x = np.arange(n_classes)
        
        # Минималистичная гистограмма
        bars = ax5.bar(x, f1_scores, alpha=0.7, color='lightblue', edgecolor='navy', linewidth=0.5)
        
        # Настройки гистограммы
        ax5.set_title('F1-score по классам (последняя эпоха)', fontsize=12, pad=10)
        ax5.set_xlabel('Классы')
        ax5.set_ylabel('F1-score')
        ax5.set_ylim(0, 1.0)
        ax5.grid(True, alpha=0.3)
        
        # Подписи классов
        ax5.set_xticks(x)
        ax5.set_xticklabels([f'{i}' for i in range(n_classes)], rotation=45 if n_classes > 30 else 0, fontsize=8)
        
        # Линия среднего значения
        mean_f1 = np.mean(f1_scores)
        ax5.axhline(y=mean_f1, color='red', linestyle='--', alpha=0.8, 
                   label=f'Средний F1: {mean_f1:.3f}')
        ax5.legend()
        
        # Добавляем значения на столбцы для лучших классов (топ-5)
        if len(f1_scores) > 0:
            # Находим индексы топ-5 классов по F1
            top_indices = np.argsort(f1_scores)[-5:][::-1]
            for idx in top_indices:
                if f1_scores[idx] > 0:
                    ax5.text(idx, f1_scores[idx] + 0.02, f'{f1_scores[idx]:.2f}', 
                            ha='center', va='bottom', fontsize=7, fontweight='bold')
    else:
        ax5.text(0.5, 0.5, 'F1 по классам недоступен', 
                ha='center', va='center', transform=ax5.transAxes, fontsize=12)
        ax5.set_title('F1-score по классам')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Graphs saved to: {save_path}")
    
    plt.show()

def calculate_pos_weight(dataset, max_weight=3.0, eps=1e-8):
    all_targets = []
    for i in range(len(dataset)):
        _, targets = dataset[i]
        all_targets.append(targets)

    all_targets = torch.stack(all_targets)
    
    positive_counts = all_targets.sum(dim=0) + eps
    negative_counts = (all_targets.size(0) - all_targets.sum(dim=0)) + eps

    freq = negative_counts / positive_counts
    pos_weight = 1 / (freq + eps)
    pos_weight = pos_weight / pos_weight.mean()

    # Нормализация:

    # Квадратный корень
    # pos_weight = torch.sqrt(pos_weight)
    # pos_weight = pos_weight / pos_weight.min()

    # ИЛИ

    # Логарифм
    # pos_weight = 1 + torch.log1p(pos_weight + eps)

    # Можно попробовать добавить
    # pos_weight = pos_weight / pos_weight.max()  # Нормализуем к [0, 1]
    # pos_weight = 1 + max_weight * pos_weight    # Масштабируем до [1, max_weight]

    # print(f"Коррекция весов: {pos_weight} \n")

    return pos_weight.to(device)



In [10]:
# ЗАГРУЗКА ДАТАСЕТА
# ----------------------------------------------------------
tags_dict = {}
# открываем подготовленный словрь
with open("../data/track_tags.pkl", 'rb') as file:
    tags_dict = pickle.load(file)

# загружаем датасет
dataset = AudioDataset("../data/datasets/128x256.h5")

# разбиваем датасет на train и test
dataset_size = len(dataset)
indices = list(range(dataset_size))

train_idx, test_idx = train_test_split(
    indices, 
    test_size = 0.3,
    train_size = 0.7,       
    random_state = 42        # для воспроизводимости
)

train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,  
    num_workers=8,   
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,  
    num_workers=4,   
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=2
)

for data, targets in train_loader:
    print(f"DATA - shape: {data.shape}")
    print(f"       min: {data.min().item():.4f}, max: {data.max().item():.4f}")
    print(f"       mean: {data.mean().item():.4f}, std: {data.std().item():.4f}")
    # print(f"target exemple: {targets[0]}")
    break


Loaded 83255 samples from HDF5
DATA - shape: torch.Size([64, 1, 128, 256])
       min: -13.8155, max: 12.1015
       mean: 0.6493, std: 4.3827


In [ ]:
#ОБУЧЕНИЕ МОДЕЛИ
def debug_model_predictions(model, loader, threshold=0.5):
    model.eval()
    
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            probabilities = torch.sigmoid(outputs)
            predictions = (probabilities > threshold).float()  # ⚡ Multi-hot вектор!
            
            print("=== MULTI-LABEL DEBUG ===")
            print(f"Outputs range: [{outputs.min():.3f}, {outputs.max():.3f}]")
            print(f"Probabilities range: [{probabilities.min():.3f}, {probabilities.max():.3f}]")
            
            # Правильная интерпретация
            print(f"Sample 0 predictions: {predictions[0].cpu().numpy()}")
            print(f"Sample 0 targets:     {targets[0].cpu().numpy()}")
            print(f"Sample 0 matched:     {((predictions[0] == targets[0]) & (targets[0] == 1)).sum().item()} correct positive tags")
            
            # Статистика по батчу
            batch_accuracy = ((predictions == targets).float().mean().item())
            positive_accuracy = ((predictions[targets == 1] == 1).float().mean().item())
            
            print(f"Batch accuracy: {batch_accuracy:.3f}")
            print(f"Positive tag accuracy: {positive_accuracy:.3f}")
            break
def debug_model_outputs(model, data_loader):
    model.eval()
    
    with torch.no_grad():
        for data, targets in data_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            
            print("=== MODEL DEBUG ===")
            print(f"Input data range: [{data.min():.3f}, {data.max():.3f}]")
            print(f"Input data mean: {data.mean():.3f}")
            
            # Проверяем выходы каждого слоя
            print(f"Model outputs range: [{outputs.min():.3f}, {outputs.max():.3f}]")
            print(f"Model outputs mean: {outputs.mean():.3f}")
            
            # Проверяем градиенты
            for name, param in model.named_parameters():
                if param.requires_grad and param.grad is not None:
                    print(f"Gradient {name}: {param.grad.abs().mean():.6f}")
                else:
                    print(f"Gradient {name}: None")
            
            break
def debug_model_weights():
    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name:30} mean: {param.data.mean().item():8.4f} std: {param.data.std().item():8.4f}")
        elif 'bias' in name:
            print(f"{name:30} mean: {param.data.mean().item():8.4f}")
 
num_epochs = 10        # количество эпох для обучения
threshold = 0.1         # порог для предсказаний
patience = 5           # остановка обучения после N эпох без улучшений

start_epoch = 1         #
history = None          # "Зашлушка" для переменных если надо будет загрузить недообученную модель
best_f1 = 0.0           #

# Инициализация модели
# model = SimpleFCModel(dataset.num_classes, 128, 256, 512)
model = AudioCNN_3(dataset.num_classes)
model.to(device)

num_param = sum(p.numel() for p in model.parameters())
output_str = f"Используется устройство: {device} \n"
output_str += f"Количество параметров модели: {(num_param / 1000000.0):.1f}M \n"

criterion = MultiLabelAsymmetricLoss(gamma_pos=4.0, gamma_neg=2.0, clip=0.05)
# criterion = nn.BCEWithLogitsLoss(calculate_pos_weight(dataset))
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# Загрузка модели
load_str = ""
# model, optimizer, history, start_epoch, best_f1, threshold, load_str = load_checkpoint(model, optimizer, "../data/models/CNN_2/best_model.pth", device)

output_str += load_str

# Запуск обучения
history, best_f1 = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=test_loader,
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=num_epochs,
    threshold = threshold,
    save_dir='../data/models/CNN_3',
    patience=patience,
    start_epoch=start_epoch,
    history=history,
    best_f1=best_f1,
    auto_threshold = True,
    output_str=output_str
)

# вывод графиков
plot_training_history(history)

debug_model_predictions(model, train_loader, threshold)
debug_model_outputs(model, train_loader)

Используется устройство: cuda 
Количество параметров модели: 1.4M 

Epoch 4/10
──────────────────────────────────────────────────
Train >>> Loss: 0.0939 | F1: 0.0766 | Precision: 0.0419 | Recall: 0.4511 | Threshold: 0.30
  Val >>> Loss: 0.0926 | F1: 0.0847 | Precision: 0.0480 | Recall: 0.4665 | Threshold: 0.30
──────────────────────────────────────────────────
Новая лучшая модель сохранена! F1: 0.0847

Epoch 5/10
──────────────────────────────────────────────────


Train >>> :  40%|████████            | 365/911 [00:32<00:49, 11.09it/s, Loss=0.0937]


KeyboardInterrupt: 

In [ ]:
#ПРОВЕРКА РАНДОМНОЙ ПЕСНИ ИЗ ПАПКИ
import torchaudio 

def random_audio_predictor(model, preprocess_function, class_names, folder_path='../data/train/', device='cuda', threshold=0.5):
    """
    Случайный выбор аудио файла с автовоспроизведением и предсказанием тегов
    """
    
    print("🎲 СЛУЧАЙНЫЙ АУДИО АНАЛИЗ")
    print("=" * 50)
    
    # Находим все аудио файлы в папке
    audio_extensions = ['*.wav', '*.mp3', '*.flac', '*.m4a', '*.ogg']
    audio_files = []
    
    for ext in audio_extensions:
        # Ищем файлы в основной папке и подпапках
        audio_files.extend(glob.glob(os.path.join(folder_path, '**', ext), recursive=True))
        audio_files.extend(glob.glob(os.path.join(folder_path, ext)))
    
    if not audio_files:
        print(f"❌ В папке {folder_path} не найдено аудио файлов!")
        return None
    
    # print(f"📁 Папка: {folder_path}")
    # print(f"📊 Найдено файлов: {len(audio_files)}")
    # print(f"🎚️  Порог: {threshold}")
    
    # Выбираем случайный файл
    selected_file = random.choice(audio_files)
    filename = os.path.basename(selected_file)
    file_path = os.path.dirname(selected_file)
    
    # print(f"\n🎯 Выбран файл: {filename}")
    # print(f"📂 Путь: {file_path}")
    # print("─" * 50)
    

    # Автовоспроизведение аудио
    print("🔊 Автовоспроизведение...")
    display(Audio(selected_file, autoplay=True))
        
    # Предсказание тегов
    probs, active_tags = quick_predict(
        model=model,
        audio_path=selected_file,
        preprocess_function=preprocess_function,
        class_names=class_names,
        device=device,
        threshold=threshold
    )
    
    print(f"Файл: {filename}")

    if probs is not None:
        # Выводим результаты
        print(f"\n🏷️  ПРЕДСКАЗАННЫЕ ТЕГИ:")
        print("─" * 30)
        
        if active_tags:
            print(f"✅ Найдено тегов: {len(active_tags)}")
            print("\n📋 Список тегов:")
            for tag in active_tags:
                prob = probs[class_names.index(tag)]
                print(f"   • {tag}: {prob:.3f}")
        else:
            print("❌ Нет тегов выше порога")
        
        # Топ-10 всех предсказаний
        top_indices = np.argsort(probs)[::-1][:10]
        print(f"\n🔝 Топ-10 всех предсказаний:")
        for i, idx in enumerate(top_indices, 1):
            prob = probs[idx]
            tag = class_names[idx]
            marker = "✅" if prob > threshold else "  "
            print(f"   {i:2d}. {marker} {tag:<25} {prob:.3f}")
        
        new_dict = {}
        for key, vals in tags_dict.items():
            new_dict[key[3:]] = vals

        print(f"\nПравильные теги: {new_dict.get(filename, "none")}")

        # # Статистика
        # print(f"\n📊 СТАТИСТИКА:")
        # print(f"   Всего возможных тегов: {len(class_names)}")
        # print(f"   Активных тегов: {len(active_tags)}")
        # print(f"   Максимальная вероятность: {max(probs):.3f}")
        # print(f"   Средняя вероятность: {np.mean(probs):.3f}")
            
        # return {
        #     'file': selected_file,
        #     'filename': filename,
        #     'probs': probs,
        #     'active_tags': active_tags,
        #     'threshold': threshold
        # }           

def preprocess_audio(audio_path, target_sr = 22050, noise = False, start=0, size=256) -> torch.Tensor:
    waveform, sr = torchaudio.load(audio_path, normalize=True, channels_first=True)
    #уменьшил частоту дискретизации, чтобы ещё меньше датасет весил, а то везде 44100 Гц или почти везде
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=22050,
        n_mels=128,        
        n_fft=2048,       
        hop_length=512,
        f_min=20,
        f_max=11025
    )(waveform)

    mel_spectrogram = torch.log(mel_spectrogram + 1e-6)

    # добавим немного шума (это скорее всего сильно замедлит подготовку данных)
    if noise:
        for x in mel_spectrogram:
            x += random.uniform(-1.0, 1.0)

    return mel_spectrogram[0,:,start:start+size]

def quick_predict(model, audio_path, preprocess_function, class_names, device='cuda', threshold=0.5):
    """
    Функция предсказания тегов для аудио файла
    """
    model.to(device)
    model.eval()
    
    try:
        # Препроцессинг
        mel_spec = preprocess_function(audio_path)
        
        # Добавляем batch dimension и перемещаем на устройство
        if len(mel_spec.shape) == 3:  # [channels, height, width]
            input_tensor = mel_spec.unsqueeze(0)  # [1, channels, height, width]
        else:
            input_tensor = mel_spec.unsqueeze(0).unsqueeze(0)  # [1, 1, height, width]
        
        input_tensor = input_tensor.to(device)
        
        # Предсказание
        with torch.no_grad():
            outputs = model(input_tensor)
            probs = torch.sigmoid(outputs).cpu().numpy()[0]
        
        # Собираем активные теги
        active_tags = []
        for i, (class_name, prob) in enumerate(zip(class_names, probs)):
            if prob > threshold:
                active_tags.append(class_name)
        
        return probs, active_tags
        
    except Exception as e:
        print(f"❌ Ошибка во время предсказания: {e}")
        return None, None

best_model = AudioCNN_2(dataset.num_classes)
optimizer = torch.optim.AdamW(best_model.parameters())
best_model,_,_,_,_,threshold, out_str = load_checkpoint(best_model, optimizer, "../data/models/CNN_2/best_model.pth", device)

print(out_str)

best_model.to(device)
best_model.eval()

# Простой вызов - один случайный файл
random_audio_predictor(
    model=best_model,
    preprocess_function=preprocess_audio,
    class_names=dataset.get_class_names(),
    folder_path='../data/test', #'/home/egr/Музыка',  # Ваша папка с аудио
    device=device,
    threshold=threshold  # Можно настроить порог
)


Загружен чекпоинт эпохи 8
Лучший F1: 0.1399
Всего эпох в истории: 8

🎲 СЛУЧАЙНЫЙ АУДИО АНАЛИЗ
🔊 Автовоспроизведение...


Файл: 268292.mp3

🏷️  ПРЕДСКАЗАННЫЕ ТЕГИ:
──────────────────────────────
✅ Найдено тегов: 3

📋 Список тегов:
   • energetic: 0.196
   • happy: 0.212
   • uplifting: 0.115

🔝 Топ-10 всех предсказаний:
    1. ✅ happy                     0.212
    2. ✅ energetic                 0.196
    3. ✅ uplifting                 0.115
    4.    corporate                 0.084
    5.    inspiring                 0.074
    6.    positive                  0.073
    7.    motivational              0.070
    8.    melodic                   0.068
    9.    summer                    0.064
   10.    advertising               0.062

Правильные теги: {'ambiental'}


/home/egr/projects/nmus/.venv/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/egr/projects/nmus/.venv/lib/python3.12/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3